# rLab Door Controller Design


## Introduction

The rLab door failed meaning that members could not access rLab using their RFID cards.
It turned out to be an issues with the actuator in the door, but whilst troubleshooting it became apparent that the control circuitry wasn't in a great state.
I have offered to create a tidyier replacement; this is the design document.

### Current System

![What a mess](img/20220615_Electronics_Overview.jpg)

The current system comprises:

* Raspberry Pi - Provides overall control, allows network access, syncs members data from the members database, and authenticates RFID card access
* Arduino - interfaces between the Pi and the 12V switching needed to operate the door
* Power Supply Board - Converts incoming DC to voltages used around the system, including battery charging for a sealed lead acid battery which ensures the door continues to operate during a power cut
* Relay Interface Board - Currently sits as a shield above the Arduino providing connectivity to the relay which provides a 12V pulse to open the door.

## Scope of the Project

### Out of Scope

* The Raspberry Pi functionality is outside the scope of this project

### In Scope

* Power supplies including to the Pi
* Battery backup and charging
* Control of the door actuator under the command of the Pi
* Add a display and/or LEDs to help troubleshooting
* Change connections to plug and socket or screw terminals, instead of random twisted wires

## Design Criteria

### Design Principles

* Aim for simplicity
* Without breaking the first principle, enable flexibility of the design
* Enable easy troubleshooting
  * Provide displays or indications of fault conditions
  * Provide serial log output

### Project Requirements

The design:

* **Must** open the door when commanded by the Raspberry Pi
* **Must** provide 5V power @ up to 1.5A to the Raspberry Pi
* **Must** continue to operate through short power cuts
* **Should** provide an LED indication and test points for each power rail
* **Should** provide an LED indication and test point for the actuator output
* **Should** include an earth stud for test point ease of use
* **Should** provide USB programming and a serial console
* **Should** make battery charge circuit adjustable so we can fine tune the charge voltage
* **Could** Provide dual footprint for SMD and through-hole indicator LEDs
* **Could** include a 16-Pin "character display" interface
* **Could** include Neopixels for better status indication
* **Could** drive the actuate output with up to 2.5A to allow for future changes to the way that the door operates
* **Could** provide connectivity for a keypad to allow access using OTP codes
* **Could** provide connectivity for an RFID card reader for potential future upgrades
* **Wont** take over the functions currently performed by the Raspberry Pi (Yet!)

### Power Supply


#### Requirements

1. **Must** provide the door actuator with 12V @ 2.5A (Quoted)
2. **Must** provide the Raspberry Pi with 5V @ 1.3A. See [Raspberry Pi Power](#raspberry-pi-power)
3. **Must** provide 3.3V @ 500mA for the ESP-32 microcontroller
4. **Must** be able to operate the door through a short power cut - implies battery backup.
5. **Could** provide additional current capability at 3.3V for peripherals

##### Mains Voltage

To make this project as safe as possible there will be no mains voltages used in the design.
DC power input will be provided by an encapsulated external power supply.

##### Power Input

The input power will need to provide at least 14V DC to charge backup batteries.
Because of dropout in the charging system it is likely that the input voltage will need to be at least 17V.
Ideal power supply voltage will be 18V-20V DC.
This range is easily achieved with laptop style power supplies.


##### Power Rails

Because of the cascading of power rails, the current requirements accumulate from the bottom up.
The quoted current capacity for each rail includes the current requirements of the rails derived from it.
The figures are very much worst case and therefore the actual supply requrements will be significantly lower.
That said, the use of a [90W, 20V supply](https://www.amazon.co.uk/DTK-Adapter-Computer-Charger-Notebook-lenovo-90w-7-9-5-5mm/dp/B07QL2TXD6) is reasonable.

* $V_{In}$ = 18-24V
  * Provided by external encapsulated power supply
  * Current requirement = 3.6A
* $V_{Batt}$ = 12V nominal (13.8V likely when charging)
  * Needed to drive the door actuator
  * Sealed lead-acid battery provides backup power on this rail
  * Current requirement = 800mA (3.3V) + 1.3A (5V) + 1.5A ($V_{Batt}$)
* $+5V$
  * Power for Raspberry Pi
  * Derived from $V_{Batt}$ using an efficient switching regulator, like [this](https://www.amazon.co.uk/dp/B08DQVJZH3)
  * Current requirement = 800mA (3.3V) + 1.3A (5.5V) = 2.1A
* $+3.3V$
  * Power for microcontroller
  * Derived from $+5V$ rail using an LDO linear regulator, such as the [AMS1117-3.3](https://octopart.com/datasheet/ams1117-3.3-ams-26604089)
  * Current requirement = 800mA

#### Raspberry Pi

##### Raspberry Pi Power

This [article](https://www.raspberrypi.com/documentation/computers/raspberry-pi.html) provides the following table giving typical power consumption figures for the Pi range:

![Raspberry Pi typical power consumption](img/Pi_Power.png)

Although the recommended power supply requirements are high, a lot of that power is to supply USB devices which could be connected to the Pi.
We will not have a large number (if any) peripherals connected and can therefore use more conservative power supply requirements.
The highest "bare board" consumption figures are for the Pi 400 @ 800mA.
An additional budget of 500mA for peripherals places the supply requirement @ 1.3A

##### Pi Header

This [guide](https://www.raspberrypi-spy.co.uk/2012/06/simple-guide-to-the-rpi-gpio-header-and-pins/?utm_source=pocket_mylist) gives a good overview of the Pi Header variations.

![40-Pin Pi GPIO Header](img/Raspberry-Pi-GPIO-Layout-Model-B-Plus.png)

We currently have an original Pi Model B, which has the 26-pin header.  
There are 2 versions of the 26-pin header with differences on pins 3, 5, 13. 
These are still GPIOs, but are numbered differently.
I want to pick GPIOs that are constant across all Pi revisions to avoid confusion in future if the Pi is swapped over.

I have chosen the following:

| Pi Header Pin | Pi GPIO | Purpose | Kicad Ref |
| --- | --- | --- | --- |
| 1 | 3V3 | Detect if Pi is connected | PI_POWERED |
| 2 | 5V | Power for the Pi | +5V |
| 6 | GND | GND | GND |
| 7 | GPIO4 | Pi Heartbeat | PI_HEARTBEAT |
| 8 | GPIO14 (TXD) | UART TX from Pi, RX to ESP | RXDI_PI |
| 10 | GPIO15 (RXD) | UART RX to Pi, TX from ESP | TXDO_PI |
| 16 | GPIO23 | Open 1 command | PI_OPEN_CMD_1 |
| 18 | GPIO24 | Open 2 command | PI_OPEN_CMD_2 |




#### Battery Charging

Sealed lead-acid batteries are the standard backup supply used in fire and security alarms.
They are well suited to trickle charging with few discharge cycles.
Lithium cells are less well suited and are not therefore being considered.

To trickle charge sealed lead-acid batteries this [article](https://www.power-sonic.com/blog/how-to-charge-a-lead-acid-battery/) suggests that you need to apply 2.3V/cell.
So for a 12V, 6 cell battery this would be in the region of 13.8V.

> The recommended constant float voltage is 2.25V-2.30V per cell.
> Maintaining this float voltage will allow the battery to define its own current level and remain fully charged without having to disconnect the charger from the battery.
> The trickle current for a fully charged battery floating at the recommended charge voltage will typically hover around the 0.001C rate (2mA for a 2Ah battery for example.)


##### High Performance Charger (UC3906)

The [datasheet](https://www.farnell.com/datasheets/45075.pdf) states that this can be configured to manage the charging and performance of lead-acid batteries, but the circuits are more complex and the solution is expensive, around £10/chip, and there seems to be a [lack of supply](https://octopart.com/search?q=uc3906) for these at present.

Although this would provide more detailed information of the charging state over I2C to the microcontroller the availability and cost mean that I am discounting this option.
Also it is probably overkill for the standby trickle charge situation where charging speed is not critical.

##### Simple option (LM317)

[TI Application Note - Battery Charging](https://www.ti.com/lit/an/snva581/snva581.pdf?ts=1656342263523)

![LM317 Battery Charging Circuit](img/LM317_Charger_2.png)

> The peak charging current or output current is controlled by
> the internal current limit of the LM317. This current limit will
> work even if a battery is connected backwards to the output
> of the charger. Should a fault condition exist for an extended
> period of time, the thermal limiting circuitry will decrease the
> output current, protecting the regulator as well as the transformer. 
> A constant voltage charger circuit is shown

This [LM317 Datasheet](https://www.ti.com/lit/ds/symlink/lm317.pdf) has the following application for constant voltage battery charging:

![LM317 Battery Charging Circuit](img/LM317_Charger.png)

$R_s$ is used to protect the battery from high current or to limit the output in case of shortcircuit.
The application note suggests $R_s = 0.2\Omega$ this would give a short circuit current of around 6A, which would then be limited by the LM317 to around 0.5A-1.5A depending on the model.
The datasheet says the following:

> When an overload occurs the device shuts down Darlington NPN output stage or reduces the output current to prevent device damage.
> The device will automatically reset from the overload.
> The output may be reduced or alternate between on and off until the overload is removed.


So the maximum power dissipation needed for $R_s$ is around 0.5W for the 1.5A version and 0.05W for the 500mA version.

$R_1$ and $R_2$ set the output voltage.
With the given values of $R1=240\Omega$ and $R2=2.4k\Omega$ the output voltage is:

$ V_{out} = V_{ref} \left(1 + \frac{R_2}{R_1}\right) $

$ V_{out} = 1.25 \left(1 + \frac{2400}{240}\right) = 13.75V  $

These values are therefore good for the trickle charge application.
However, because of potential tolerances in the LM317 ($V_{ref}$) it would be sensible to make $R2 = 3K\Omega$ variable resistor.

These ICs are around £1.50 each and currently seem to be [available](https://octopart.com/lm317p-stmicroelectronics-530526?r=sp) to purchase.


In [ ]:
R1 = 220
R2 = 2200
Tol = 0.01
VRef = 1.25
VOut = VRef * (1 + R2/R1)
print(f"The output voltage is: {VOut}V")

The output voltage is: 13.75V


#### Circuit Protection

##### Level Shifter

Most of the connected devices are 5V native.
The ESP32 and Pi are 3V3 devices.
Level shifters are recommended on the 5V inputs and outputs.
A bidirectional MOSFET level shifter from [this page](http://www.penguintutor.com/electronics/mosfet-levelshift) is shown:

![Bidirectional MOSFET level shifter](img/bidirectional_mosfet_level.jpeg)

I have chosen to use the 2N7002 MOSFET as this is a basic part in JCLPCB.


##### 3V3 Supply Through Programmer

According to [this article](https://electronics.stackexchange.com/questions/201876/damaging-a-linear-regulator-applying-a-voltage-to-the-output) feeding 3V3 to the output of the Linear Regulator without a higher voltage at the input is not recommended.
This could happen if 3V3 was supplied by a serial programmer without the system being otherwise powered.
I think that this is another version of reverse polarity protection?

A simple solution is to place a Schottky diode on the output of the linear regulator, but this will drop the output voltage by 0.2V to around 3.1V.
This should not be a problem as the ESP32 can accept a minimum voltage of 3.0V.
If we can't accept that voltage drop then we can look at using an ideal diode as suggested in [this post](https://www.powerelectronictips.com/inexpensive-ideal-diode-mosfet-circuit/)

##### Reverse polarity Protection

In the [Battery Charging](#battery-charging) section above, it states that the LM317 will protect itself and its supply from a reverse connected battery or other failures.

To fix downstream problems place a Schottky diode between the battery and follow-on circuits.

Need to consider "MOSFET as a diode" type protection from [this page](https://www.rs20.net/w/2013/02/using-mosfets-as-blocking-diodes-reverse-polarity-protection).

##### Fuses

Safety of last resort.
Protect input, battery and 2 x 12V actuator outputs with a fuse.

## USB-C connector

This [article](https://pcbartists.com/design/embedded/how-to-replace-microusb-with-usb-c-connector/) shows the following schematic for using a USB-C connector with USB 2.0 protocol.
Although it uses a CP2102 instead of the CH340G it shows how the connections from the USB-C should be handled.

![USB-C Connector for USB 2.0 Schematic](img/USB-C_2.0.png)


## Thoughts from the design meeting 

I met with Rupert and Scott on 21 Jun 22.  These were the additional design points captured from that discussion:

* WG26 (Protocol) outdoor security keypad
* Tamper input
* 6 Neopixels for door confirmation
* Provide second MOSFET switched output
* Internal pushbutton for door actuate testing
* Fuses on 12V output?
* 12V Input via battery connector?
* Level converter to 5V for neopixels
* 2 x neopixels output?
* Dual ESP?
* Break out spare GPIO including some to the Pi header